In [1]:
import numpy as np
import pandas as pd
import json
import os
import re

## load both USA and DC dataset

In [5]:
# get all json file path in the folder
folder_path = "../dataset/USA"

file_paths = []
for filename in os.listdir(folder_path):
    filepath = os.path.join(folder_path, filename)
    if os.path.isfile(filepath):
        file_paths.append(filepath)

len(file_paths)

47

In [6]:
folder_path = "../dataset/DC"

for filename in os.listdir(folder_path):
    filepath = os.path.join(folder_path, filename)
    if os.path.isfile(filepath):
        file_paths.append(filepath)

len(file_paths)

85

## Extract job types

In [7]:
job_results_by_types = {}

for file_path in file_paths:
    match = re.search(r'/([^/]+)-\d+\.json$', file_path)
    if match:
        job_type = match.group(1)
        with open(file_path, 'r') as f:
            data = json.load(f)
        job_results_by_types[job_type] = job_results_by_types.get(job_type, []) + [data,]

print(job_results_by_types.keys())

dict_keys(['block-chain', 'data-scientist', 'natural-language-processing', 'reinforcement-learning', 'neural-networks', 'big-data-and-cloud-computing', 'data-analyst', 'machine-learning', 'time-series-analysis', 'deep-learning', 'time-series'])


In [9]:
salary_by_types = pd.DataFrame(columns=['type', 'salary', 'job_id', 'description', 'in_DC'])
data = []

## get dmv locations

In [10]:
dmv_locations = ['dc', 'md', 'va', 'maryland', 'virginia', 'washington']

## find average salary of each job types not in dc

In [11]:
for job_type in job_results_by_types:
    datasets = job_results_by_types[job_type]

    for dataset in datasets:
        if 'jobs_results' not in dataset:
            continue 
        jobs_results = dataset["jobs_results"]
        for job_result in jobs_results:
            description = job_result["description"] 
            job_highlights = job_result["job_highlights"] 

            location = job_result['location']

            location_in_dmv = False 
            for dmv_location in dmv_locations:
                if dmv_location in location.lower():
                    location_in_dmv = True
                    break 

            if location_in_dmv:
                continue 

            qualifications = None 
            responsibilities = None 
            benefits = None 

            for highlight in job_highlights:
                if highlight.get("title") == "Qualifications":
                    qualifications = highlight.get("items") 
                elif highlight.get("title") == "Responsibilities":
                    responsibilities = highlight.get("items")
                elif highlight.get("title") == 'Benefits':
                    benefits = highlight.get("items") 

            job_id = job_result["job_id"]
            salary_regex = r"\$([0-9,]+)"

            list_of_text = [description, qualifications, responsibilities, benefits]
            total_salary = 0
            num_salary = 0
            for text_item in list_of_text:
                if text_item is None:
                    continue 
                else:
                    if type(text_item) is list:
                        for text in text_item:
                            matches = re.findall(salary_regex, text)

                            if len(matches) == 0:
                                continue 
                            
                            for match in matches:
                                salary = float(match.replace(',', ''))
                                if salary < 100000:
                                    continue 
                                total_salary += salary
                                num_salary += 1
                    else:
                        matches = re.findall(salary_regex, text_item)

                        if len(matches) == 0:
                            continue
                        
                        for match in matches:
                            salary = float(match.replace(',', ''))
                            if salary < 100000:
                                continue 
                            total_salary += salary
                            num_salary += 1

            if num_salary != 0:
                avg_salary = total_salary / num_salary
                data.append({'type': job_type, "salary": avg_salary, 'job_id': job_id, 'description': description, 'in_DC': location_in_dmv})


salary_by_types = salary_by_types.append(data, ignore_index=True)

/var/folders/h0/sq_brb3d0_lfvtk4fhlbrck00000gn/T/ipykernel_51001/2759187651.py:76: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  salary_by_types = salary_by_types.append(data, ignore_index=True)


In [12]:
salary_by_types.head()

,type,salary,job_id,description,in_DC
0,block-chain,156000.0,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIFx1MDAyNiBEaW...,We are expanding Deloitte's Audit & Assurance ...,False
1,block-chain,180000.0,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEVuZ2luZWVyIi...,We are seeking a highly motivated and skilled ...,False
2,block-chain,500000.0,eyJqb2JfdGl0bGUiOiJEZUZpIEJsb2NrY2hhaW4gQ28tRm...,Cryptops is currently looking for a Co-Founder...,False
3,block-chain,180000.0,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEVuZ2luZWVyIi...,21.co is the world's leader in providing acces...,False
4,block-chain,150000.0,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEVuZ2luZWVyIi...,Jump Crypto is committed to building and stand...,False


In [13]:
salary_by_types = salary_by_types.drop_duplicates(subset=['job_id', 'salary'])

In [14]:
salary_by_types.head(5) 

,type,salary,job_id,description,in_DC
0,block-chain,156000.0,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIFx1MDAyNiBEaW...,We are expanding Deloitte's Audit & Assurance ...,False
1,block-chain,180000.0,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEVuZ2luZWVyIi...,We are seeking a highly motivated and skilled ...,False
2,block-chain,500000.0,eyJqb2JfdGl0bGUiOiJEZUZpIEJsb2NrY2hhaW4gQ28tRm...,Cryptops is currently looking for a Co-Founder...,False
3,block-chain,180000.0,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEVuZ2luZWVyIi...,21.co is the world's leader in providing acces...,False
4,block-chain,150000.0,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEVuZ2luZWVyIi...,Jump Crypto is committed to building and stand...,False


In [15]:
salary_by_types.shape

(153, 5)

In [16]:
salary_by_types.groupby("type")['salary'].mean()

type
big-data-and-cloud-computing    199083.333333
block-chain                     212631.125000
data-analyst                    123714.285714
data-scientist                  159147.500000
deep-learning                   205105.625000
machine-learning                193466.209150
natural-language-processing     171844.464286
neural-networks                 230906.250000
reinforcement-learning          194902.869048
Name: salary, dtype: float64

In [17]:
salary_by_types.to_csv('../dataset/salary_not_in_dc.csv', index=False)